In [7]:
import chardet

with open("datasets/foodstruct_nutritional_facts.csv", "rb") as f:
    raw_data = f.read(100000)  
    result = chardet.detect(raw_data)

print("Detected encoding:", result["encoding"])

Detected encoding: utf-8


In [8]:
import pandas as pd

df = pd.read_csv("datasets/foodstruct_nutritional_facts.csv", encoding="utf-8", encoding_errors="replace") 

print(df.head(10))

       Food Name Category Name  Calcium  Calories  Carbs  Cholesterol  \
0        Acerola        Fruits    0.012      32.0    7.7          0.0   
1          Apple        Fruits    0.006      52.0   14.0          0.0   
2        Apricot        Fruits    0.013      48.0   11.0          0.0   
3    Dried fruit        Fruits    0.055     241.0   63.0          0.0   
4        Avocado        Fruits    0.012     160.0    8.5          0.0   
5         Banana        Fruits    0.005      89.0   23.0          0.0   
6     Blackberry        Fruits    0.029      43.0    9.6          0.0   
7      Blueberry        Fruits    0.006      57.0   14.0          0.0   
8  Boysenberries        Fruits    0.027      50.0   12.0          0.0   
9     Breadfruit        Fruits    0.017     103.0   27.0          0.0   

    Copper   Fats  Fiber    Folate  ...  Vitamin D  Vitamin E     Vitamin K  \
0  0.00009   0.30    1.1  0.000014  ...        NaN        NaN           NaN   
1  0.00003   0.17    2.4  0.000003  ..